In [16]:
import os
import sys
from pathos.pools import ThreadPool, ProcessPool
from functools import partial
sys.path.append(os.path.join(os.getcwd(), '..'))
from src.funcs.db_io import (create_db_connection, read_table_to_df)
from configs import db_config
import numpy as np 
from tqdm import tqdm
from heapq import nsmallest
from src.funcs.data_io import (get_all_file_paths_in_dir, process_site_table)
from src.funcs.db_io import append_data_to_table, tensor_aws_db1_url
from src.funcs.decorators import timer

In [2]:
data_path = "/Users/vasu/Library/CloudStorage/OneDrive-DecisionTreeAnalyticsServices/TensorDynamics/Data/Satellite"

In [3]:
db_connection = create_db_connection(dbname=db_config.dbname,
                                     host=db_config.host,
                                     port=db_config.port,
                                     user=db_config.user,
                                     password=db_config.password)

In [4]:
locations_df = read_table_to_df(con=db_connection,
                                table_str=db_config.site_table,
                                schema=db_config.site_table_schema)

locations_dict = process_site_table(site_df=locations_df, site_type=None)

In [5]:
nc_file_paths = get_all_file_paths_in_dir(folder_path=data_path,
                                          find_match='S_NWC_CT',
                                          ignore_matches=None)

In [3]:
sample_file = nc_file_paths[0]

NameError: name 'nc_file_paths' is not defined

In [1]:
"""
Class to extract data from a NETCDF file
"""
import re

import xarray as xr
import pandas as pd
import haversine


class SatelliteNCExtractor:

    def __init__(self,
                 nc_file_path,
                 locations_dict=None,
                 lat_col=None,
                 lon_col=None,
                 exclude_cols_str_matching=None,
                 extract_attrs=None):
        """
        Extracts dataframe from NETCDF File with given params

        :param nc_file_path: File path to read
        :type nc_file_path: str, path like

        :param locations_dict: locations dictionary to filter netcdf file variables. Optional, default to None.
            Will return all lat lon info if None.
        :type locations_dict: dict

        :param lat_col: Latitude index label in netcdf file. Optional, defaults to None - will try to infer
        :type lat_col: str

        :param lon_col: Longitude index label in netcdf file. Optional, defaults to None - will try to infer
        :type lon_col: str

        :param time_col: Time column index in netcdf file. Optional, defaults to None - will try to infer
        :type time_col: str

        :param exclude_cols_str_matching: List of string matches to avoid in output. Can be part of the column names.
            Optional, defaults to ['bnds']. i.e. all column labels with 'bnds' will be omitted.
        :type exclude_cols_str_matching: [str], list of str

        :param extract_attrs: List of attribute names of the netcdf file to extract.
            Defaults to ['title', 'experiment', 'creation_date']
        :type lat_col: [str], list of str
        """
        self.file_path = nc_file_path
        self.locs_dict = locations_dict
        self.lat_col = lat_col
        self.lon_col = lon_col
        self.exclude_cols_str_matching = exclude_cols_str_matching or ['bnds']
        self.global_attrs = extract_attrs or ['keywords', 
                                              'date_created',
                                              'time_coverage_start', 
                                              'time_coverage_end',
                                              'product_name',
                                              'nominal_product_time',
                                              'satellite_identifier', 
                                              'id', 
                                              'summary', 
                                              'title']

    @staticmethod
    def calc_haversine_dist(loc1, loc2, unit='KM') -> float:
        """
        Calculates haversine (great-circle) distance between two co-ordinates in km
        :param loc1: location 1 (lat1, lon1)
        :param loc2: location 2 (lat2, lon2)
        :param unit: UNITS of distance, optional, defaults to KM. Options = [KM, MI] i.e. [kilometers, miles]
        :return: distance in unit
        :rtype: float
        """
        if not isinstance(loc1, tuple) or not isinstance(loc2, tuple):
            raise TypeError("Cannot calculate distance. Please provide location as (lat, lon) ")

        if 'km' in unit.lower():
            unit = haversine.Unit.KILOMETERS
        elif 'mi' in unit.lower():
            unit = haversine.Unit.MILES
        else:
            raise NotImplementedError("Only 'km' or 'mi' are implemented units while calculating haversine distance.")
        return haversine.haversine(point1=loc1, point2=loc2, unit=unit)
    
    @staticmethod
    def haversine_np(lon1, lat1, lon2, lat2):
        """
        Calculate the great circle distance between two points
        on the earth (specified in decimal degrees)

        All args must be of equal length.    

        """
        lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

        c = 2 * np.arcsin(np.sqrt(a))
        km = 6367 * c
        return km
    
    @staticmethod
    def drop_columns_matching(data_frame, col_matches) -> pd.DataFrame:
        """
        Drops columns from the dataframe that matches any given str in the iter of col_matches
        """
        temp_df = data_frame.copy()
        drop_cols = [col for col in data_frame.columns if any(x.lower() in col.lower() for x in col_matches)]
        return temp_df.drop(drop_cols, axis=1)

    @staticmethod
    def get_attrs_from_xarray(xarray_obj, attr_names) -> dict:
        """Extract attributes from NC file with the given attr_names"""
        return {name: xarray_obj.attrs.get(name) for name in attr_names}

    @staticmethod
    def add_dict_items_to_df(data_frame, attr_dict, prefix=None):
        temp_df = data_frame.copy()
        counter = 0
        for attr, value in attr_dict.items():
            name = f"{prefix}_{attr}" if prefix else attr
            temp_df.insert(loc=counter, column=name, value=value)
            counter += 1
        return temp_df

    @staticmethod
    def get_timezone(xarray_obj):
        return str(pd.to_datetime(xarray_obj.attrs.get('date_created', None)).tz)
    
    @staticmethod
    def replace_special_chars(my_string, replace_with='_') -> str:
        """
        Replaces special characs from a string with regex.
        """
        return re.sub('[^A-Za-z0-9]+', replace_with, my_string).lower()

    @staticmethod
    def extract_all_vars(xr_obj):
        output_df = pd.DataFrame()
        var_names = list(xr_obj.variables.keys())
        for var in var_names:
            var_obj = xr_obj.variables.get(var)
            if var_obj is not None:
                melted_var = pd.DataFrame({var: var_obj.values.ravel()})
                output_df = pd.concat([output_df, melted_var], axis=1)
        return output_df

    @staticmethod
    def convert_time_index_to_datetime(data_frame, time_col):
        temp_df = data_frame.copy()
        temp_df[time_col] = pd.to_datetime(temp_df[time_col])
        # temp_df[time_col] = temp_df[time_col].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
        # temp_df[time_col] = pd.to_datetime(temp_df[time_col], format='%Y-%m-%d %H:%M:%S')
        return temp_df

    @staticmethod
    def extract_cords(xr_obj, lat_col, lon_col):
        melted_lats = pd.DataFrame({lat_col: xr_obj.coords[lat_col].values.ravel()})
        melted_lons = pd.DataFrame({lon_col: xr_obj.coords[lon_col].values.ravel()})
        return pd.concat([melted_lats, melted_lons], axis=1)
    
    @staticmethod
    def find_k_closest_number(find_number, search_array, k):
        return nsmallest(k, search_array, key=lambda x: abs(x - find_number))
    
    def open_xarray_obj(self):
        """Opens xarray dataframe from netcdf file"""
        return xr.open_dataset(self.file_path)

    def add_timezone(self, data_frame, xarray_obj):
        temp_df = data_frame.copy()
        temp_df['tz'] = self.get_timezone(xarray_obj=xarray_obj)
        return temp_df
    
    def clean_col_names(self, data_frame) -> pd.DataFrame:
        """Clean column names from special characters"""
        temp_df = data_frame.copy()
        temp_df.columns = [self.replace_special_chars(col) for col in temp_df.columns]
        return temp_df

    def find_closest_distance_haversine_np(self, locations_dict, lat_lon_df, lat_col='lat', lon_col='lon'):
        output_dict = {}
        for location in locations_dict:
            site_actual_location = locations_dict.get(location)
            site_lat = site_actual_location[0]
            site_lon = site_actual_location[1]

            temp_df = lat_lon_df.copy()
            temp_df['site_lat'] = site_lat
            temp_df['site_lon'] = site_lon
            temp_df['distance'] = self.haversine_np(lon1=temp_df[lon_col],
                                                    lat1=temp_df[lat_col],
                                                    lon2=temp_df['site_lon'],
                                                    lat2=temp_df['site_lat'])
            required_row = temp_df.loc[temp_df['distance'].idxmin()]
            required_lat = required_row.loc[lat_col]
            required_lon = required_row.loc[lon_col]
            required_distance = required_row.loc['distance']
            output_row = {location: {'NC': (required_lat, required_lon), 'SITE': site_actual_location}}
            output_dict.update(output_row)
        return output_dict

    def get_closest_nc_distances(self, locations_dict, xarray_df, lat_col, lon_col) -> dict:
        """
        Prepares a dictionary of actual site lat lons and the closes lat lons in a dataframe
        :param locations_dict: Locations dict in format {site : (lat, lon), site2: (lat2, lon2). .. .}
        :param xarray_df: dataframe containing lat lon cols to search from
        :param lat_col: column label containing lats
        :param lon_col: column label containing lons
        :return: dictionary of actual site lat lons and the closes lat lons found
            {'site: {'SITE': (lat, lon), 'NC':(lat`, lon`)}, 'site2':{'SITE': (lat, lon), 'NC':(lat`, lon`)} ... }
        :rtype: dict
        """
        lat_lon_df = xarray_df[[lat_col, lon_col]].drop_duplicates().reset_index(drop=True).copy()
        output_dict = self.find_closest_distance_haversine_np(locations_dict=locations_dict, 
                                                              lat_lon_df=lat_lon_df,
                                                              lat_col=lat_col,
                                                              lon_col=lon_col)
        return output_dict

    def filter_xarray_df_for_locs(self, xarray_df, location_dict, lat_col, lon_col) -> pd.DataFrame:
        """
        Filter xarray dataframe for incoming locations
        :param xarray_df: xarray df
        :param location_dict: Locations dict in format {site : (lat, lon), site2: (lat2, lon2). .. .}
        :param lat_col: col label containing latitude
        :param lon_col: col label containing longitude
        :return: netcdf dataframe filtered for locations
        """
        # GET AVAIL LOCATIONS
        locations_dict_nc = self.get_closest_nc_distances(locations_dict=location_dict,
                                                          xarray_df=xarray_df,
                                                          lat_col=lat_col,
                                                          lon_col=lon_col)
        # ITER OVER SITES
        file_df = pd.DataFrame()
        for location_name, location_meta in locations_dict_nc.items():
            actual_site_loc = location_meta['SITE']
            nc_site_loc = location_meta['NC']

            loc_df = xarray_df[(xarray_df[lat_col] == nc_site_loc[0]) &
                               (xarray_df[lon_col] == nc_site_loc[1])].copy().reset_index(drop=True)
            loc_df['site_name'] = location_name
            loc_df['site_lat'] = actual_site_loc[0]
            loc_df['site_lon'] = actual_site_loc[0]
            loc_df['distance_site_grid_point_km'] = self.calc_haversine_dist(loc1=actual_site_loc,
                                                                             loc2=nc_site_loc,
                                                                             unit='KM')
            file_df = pd.concat([file_df, loc_df], axis=0)
        return file_df.reset_index(drop=True)

    def get_data(self) -> pd.DataFrame:
        """
        wrapper for:
        1. Conversion process from NETCDF TO XARRAY
        2. Extracting attributes & dataframe
        3. Filtering for locations
        4. Cleaning of column names and dropping extra columns
        """
        xarray_obj = self.open_xarray_obj()  # OPEN X ARRAY OBJECT
        xarray_attrs = self.get_attrs_from_xarray(xarray_obj=xarray_obj,
                                                  attr_names=self.global_attrs)  # EXTRACT ATTRIBUTES
        
        xarray_df = self.extract_all_vars(xr_obj=xarray_obj)
        if self.locs_dict:
            xarray_df = self.filter_xarray_df_for_locs(xarray_df=xarray_df,
                                                       location_dict=self.locs_dict,
                                                       lat_col=self.lat_col,
                                                       lon_col=self.lon_col)

        xarray_df = (xarray_df
                     .pipe(self.drop_columns_matching, self.exclude_cols_str_matching)  # EXCLUDE COLS (Optional)
                     .pipe(self.add_timezone, xarray_obj)  # ADD TIME ZONE IF PRESENT  (FROM 'date_created' attr)
                     .pipe(self.add_dict_items_to_df, xarray_attrs)  # ADD ATTRIBUTES FROM NETCDF ATTRIBUTES
                     .pipe(self.convert_time_index_to_datetime, 'nominal_product_time')  # CONVERT COL TO DATETIME LIKE
                     .pipe(self.convert_time_index_to_datetime, 'date_created')  # CONVERT COL TO DATETIME LIKE
                     .pipe(self.convert_time_index_to_datetime, 'time_coverage_start')  # CONVERT COL TO DATETIME LIKE
                     .pipe(self.convert_time_index_to_datetime, 'time_coverage_end')  # CONVERT COL TO DATETIME LIKE
                     .pipe(self.clean_col_names)  # REMOVE SPECIAL CHARS FROM COL NAMES
                     )
        return xarray_df


In [21]:
for file in tqdm(nc_file_paths):
    time_stamp_data = SatelliteNCExtractor(nc_file_path=file, 
                                           locations_dict=locations_dict, 
                                           lat_col='lat',
                                           lon_col='lon', 
                                           exclude_cols_str_matching=['nx', 'ny']).get_data()

 18%|██████████████████████████████▎                                                                                                                                      | 104/566 [02:28<11:00,  1.43s/it]

KeyboardInterrupt



In [125]:
# master_flag_names = pd.DataFrame()
# for variable in ['ct', 'ct_cumuliform', 'ct_multilayer', 
#                  #'ct_status_flag',
#                  #'ct_conditions',
#                  #'ct_quality'
#                 ]:
#     ct_variable = xr_obj.variables[variable]
#     variable_long_name = ct_variable.attrs.get('long_name')
#     variable_range = ct_variable.attrs.get('valid_range')
#     variable_range = f"{min(variable_range)} - {max(variable_range)}"
#     flag_values = ct_variable.attrs.get('flag_values')
#     flag_meanings = ct_variable.attrs.get('flag_meanings').split(' ')
    
#     # VARIABLE META DF
#     variable_meta_df = pd.DataFrame({'flag_value': flag_values, 
#                                      'flag_meaning': flag_meanings,})
#     variable_meta_df.insert(loc=0, column='valid_range', value=variable_range)
#     variable_meta_df.insert(loc=0, column='variable_long_name', value=variable_long_name)
#     variable_meta_df.insert(loc=0, column='variable', value=variable)
    
#     master_flag_names = pd.concat([master_flag_names, variable_meta_df], axis=0)
# master_flag_names.reset_index(drop=True, inplace=True)

In [126]:
# append_data_to_table(data=master_flag_names,
#                      db_url=tensor_aws_db1_url(),
#                      table_name='ct_meta_info',
#                      schema='td_satellite')